# Libs

In [25]:
from  openai import OpenAI # type: ignore
import minsearch # type: ignore
import json
import textwrap
from elasticsearch import Elasticsearch
from elasticsearch import NotFoundError
from tqdm.auto import tqdm

# Api key

In [26]:
apikey = open('txt.txt').read()
client = OpenAI(api_key=apikey)

# Search by prepared search engine

In [27]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

documents = []
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [28]:
index = minsearch.Index(
        text_fields = ['question', 'text','section'],
        keyword_fields = ['course'],
)
index.fit(documents)

In [29]:
q = 'the course has already started, can I still enroll?'

boost = {'question': 3., 'section': 0.5 }
results = index.search(
            query = q,
            filter_dict = {'course': 'data-engineering-zoomcamp'},
            boost_dict = boost,
            num_results = 5
)
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [30]:
response = client.chat.completions.create(
                model='gpt-4o',
                messages = [{'role':'user', 'content':q}]
)

In [31]:
textwrap.wrap(response.choices[0].message.content, 50)

['It depends on the specific course and the policies',
 'of the institution offering it. Here are a few',
 'steps you can take:  1. **Check the Course',
 'Details:**    - Review the course syllabus or',
 'description provided online for any policies on',
 'late enrollment.  2. **Contact the Instructor:**',
 '- Reach out to the course instructor or professor',
 'directly to ask if late enrollment is possible and',
 'what steps you might need to take.  3. **Speak',
 'with the Registrar:**    - Contact your',
 'institution’s registrar or admissions office to',
 'understand the formal process for enrolling in a',
 'course that has already begun.  4. **Review',
 'Deadlines:**    - Check if there is a formal',
 'deadline for adding or dropping courses, which is',
 'often published in the academic calendar.  5.',
 '**Explain Your Situation:**    - If you have a',
 'compelling reason for missing the start of the',
 'course, explain your situation politely when you',
 'contact the instructor or

# Gen answer w/ OpenAI + context

In [32]:
prompt_template = """ 
You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT.
Use only the facts from the CONTEXT when answering
the QUESTION.
If the CONTEXT doesnt contain the answer, output NONE.

QUESTION: {question}

CONTEXT: {context}

""".strip()

context = ''

for doc in results:
    context = (context + 
    f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n")

prompt = prompt_template.format(question=q, context=context).strip()
# print(prompt)

In [33]:
response = client.chat.completions.create(
                model='gpt-4o',
                messages = [{'role':'user', 'content':prompt}]
)
textwrap.wrap(response.choices[0].message.content, 50)

["Yes, even if you don't register, you're still",
 'eligible to submit the homeworks. Be aware,',
 'however, that there will be deadlines for turning',
 "in the final projects. So don't leave everything",
 'for the last minute.']

# The RAG flow

In [34]:
# RAG - retrival augmented generation
def search(query):
    boost = {'question': 3., 'section': 0.5 }
    results = index.search(
                query = query,
                filter_dict = {'course': 'data-engineering-zoomcamp'},
                boost_dict = boost,
                num_results = 10
    )
    return results

def build_prompt(query, search_results):
    prompt_template = """ 
    You are a course teaching assistant. 
    Answer the QUESTION based on the CONTEXT.
    Use only the facts from the CONTEXT when answering
    the QUESTION.
    If the CONTEXT doesnt contain the answer, output NONE.

    QUESTION: {question}

    CONTEXT: {context}

    """.strip()

    context = ''

    for doc in search_results:
        context = (context + 
        f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n")

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
                    model='gpt-4o',
                    messages = [{'role':'user', 'content':prompt}]
    )
    return response.choices[0].message.content

In [35]:
query = 'the course already started. can i enroll?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
answer = rag(query)
textwrap.wrap(answer,50)

['Yes, you can still join the course after the start',
 'date. Be aware, however, that there will be',
 'deadlines for turning in the final projects.']

# Search with Elasticsearch

In [36]:
es_client = Elasticsearch('http://localhost:9200')

In [37]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'
# es_client.indices.create(index=index_name, body=index_settings)
try:
    es_client.indices.get(index=index_name)
    print(f"{index_name} already exists")
    # es.indices.delete(index=INDEX_NAME, ignore=[400, 404]) <- uncomment if you want to remove the index.
except NotFoundError:
    response = es_client.indices.create(index=index_name, body=index_settings)
    print(response)

course-questions already exists


In [38]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document = doc)

100%|██████████| 948/948 [00:23<00:00, 40.29it/s]


In [39]:
query = 'I just discovered the course. Can I still join it?'

In [40]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }
    response = es_client.search(index=index_name, body=search_query)
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])

    return result_docs

elastic_search(query)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the 

In [41]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer
answer = rag(query)

In [42]:
textwrap.wrap(answer,50)

["Yes, even if you don't register, you're still",
 'eligible to submit the homeworks. Be aware,',
 'however, that there will be deadlines for turning',
 "in the final projects. So don't leave everything",
 'for the last minute.']